In [1]:
import pandas as pd
import numpy as np
import filecmp
import os
import re
import sys
import string
working_dir = 'D:\File comparision'
source_path = working_dir + '\Source'
target_path = working_dir + '\Target'
source_processed_path = working_dir + '\SourceProcessed'
target_processed_path = working_dir + '\TargetProcessed'

In [2]:
os.chdir(source_path)
for filename in os.listdir(os.getcwd()):
    with open(filename) as f:
        with open(source_processed_path + '/' + filename , "w") as f1:
            for line in f:
                if (not '*' in line and line.strip()):
                    f1.writelines(line) 

In [3]:
os.chdir(target_path)
for filename in os.listdir(os.getcwd()):
    with open(filename) as f:
        with open(target_processed_path + '/' + filename , "w") as f1:
            for line in f:
                if (not '*' in line and line.strip()):
                    f1.writelines(line) 

In [4]:
def file_comparison(dir1,dir2):
    os.chdir(working_dir)
    orig_stdout = sys.stdout
    f = open('out.txt', 'w')
    sys.stdout = f

    for i in range(1):
        print ('i = ', filecmp.dircmp(dir1, dir2).report_full_closure())

    sys.stdout = orig_stdout
    f.close()

    file1 = open("out.txt","r+")

    data = file1.readlines()
    if len(data)>1:
        data1 = data[1].partition("[")[2].partition("]")[0].split(',')
        data1_header = data[1].split(':')[0]
        df1 = pd.DataFrame(data1, columns=[data1_header])
        output1 = pd.concat([pd.DataFrame([data1_header] * df1.shape[0], columns=['Id']),pd.DataFrame(data1, columns=['Value'])], axis=1, sort=False)
    else:
        output1 = pd.DataFrame([])

    if len(data)>2:
        data2 = data[2].partition("[")[2].partition("]")[0].split(',')
        data2_header = data[2].split(':')[0]
        df2 = pd.DataFrame(data2, columns=[data2_header])
        output2 = pd.concat([pd.DataFrame([data2_header] * df2.shape[0], columns=['Id']),pd.DataFrame(data2, columns=['Value'])], axis=1, sort=False)
    else:
        output2 = pd.DataFrame([])

    if len(data)>3:
        data3 = data[3].partition("[")[2].partition("]")[0].split(',')
        data3_header = data[3].split(':')[0]
        df3 = pd.DataFrame(data3, columns=[data3_header])
        output3 = pd.concat([pd.DataFrame([data3_header] * df3.shape[0], columns=['Id']),pd.DataFrame(data3, columns=['Value'])], axis=1, sort=False)
    else:
        output3 = pd.DataFrame([])

    if len(data)>4:
        data4 = data[4].partition("[")[2].partition("]")[0].split(',')
        data4_header = data[4].split(':')[0]
        df4 = pd.DataFrame(data4, columns=[data4_header])
        output4 = pd.concat([pd.DataFrame([data4_header] * df4.shape[0], columns=['Id']),pd.DataFrame(data4, columns=['Value'])], axis=1, sort=False)
    else:
        output4 = pd.DataFrame([])

    

    
    output = pd.concat([output1, output2, output3, output4])
    
    return output

    

In [5]:
os.chdir(working_dir)
output = file_comparison('SourceProcessed', 'TargetProcessed')

In [6]:
output['Value'] = output['Value'].str.replace('\'','')

In [7]:
output = output.rename(columns={'Id': 'Comparison Result Category', 'Value' : 'File Name'})
output.to_csv('file_comparison_result.csv', index=False)

In [8]:
output_summary= output.groupby('Comparison Result Category')['File Name'].count().reset_index()

In [9]:
output['Available in Source'] = output['Comparison Result Category'].apply(lambda x : 0 if x=='Only in TargetProcessed ' else 1)
output['Available in Target'] = output['Comparison Result Category'].apply(lambda x : 0 if x=='Only in SourceProcessed ' else 1)
output['Target Mismatch'] = output['Comparison Result Category'].apply(lambda x : 0 if x=='Identical files ' else 1)

In [10]:
output.to_csv('File_Comparison_Result.csv', index=False)